# Import FallDetection module

In [1]:
# 모듈로 만들어서 import해서 사용하기
class FallDetection:
    def __init__(self, buffer_size, fall_threshold, long_lie_threshold):
        self.buffer_size = buffer_size
        self.fall_threshold = fall_threshold
        self.long_lie_threshold = long_lie_threshold
        
        self.buffer = []
        self.long_lie_window = []
        self.lying_cnt = 0
        
        
        self.STANDING = 1
        self.BENDING = 2
        self.LYING = 3
       

    
    def buffer_step(self, label):
        #self.lying_cnt = 0
        self.buffer.append(label)
        
        if len(self.buffer)>self.buffer_size:
            self.buffer.pop(0)
            

    def detect_fall(self):
        # st: standing timestamp
        # lt: lying timestamp
        for st, label in enumerate(self.buffer):
            if label==self.STANDING:
                for lt in range(st, st+self.fall_threshold):
                    if lt>len(self.buffer)-1:
                        break
                    if self.buffer[lt] == self.LYING:
                        self.st = st
                        self.lt = lt
                        return True
        
        return False
    
    
    def detect_long_lie(self):
        
        self.lying_cnt = 0
        self.long_lie_window = []
        
        #1. declaring sliding window
        for t in range(self.lt, self.lt+self.long_lie_threshold):
            if t>len(self.buffer)-1:
                break
            
            self.long_lie_window.append(self.buffer[t])
            
            # initiate lying count
            if self.buffer[t] == self.LYING:
                self.lying_cnt +=1
        
        # alarm condition
        if self.lying_cnt >= self.long_lie_threshold:
            return True
        
        else:
            return False
        

    
    def generate_alarm(self):
        print("[ALERT] fall-down has just occurred!")
        print("fall detected between "+ str(self.st) + " and "+ str(self.lt) )
        print("self.buffer: ", self.buffer)
        print("--------")
        
        

# Class Instance

## Assumption in this example
- fall: 5 frames
- long-lie: 7 frames

## Heuristic in reality
- fall: 0.33 seconds
- long-lie: 30 seconds

In [2]:
test = FallDetection(20, 5, 7)

# Generate Stream

In [3]:
import numpy as np

In [4]:
stream = [np.random.choice([1,2,3]) for idx in range(10000)] 

# Detect Falldown

In [5]:
for label in stream:
    test.buffer_step(label)
    #print(test.buffer)
    if test.detect_fall():
        if test.detect_long_lie():
            test.generate_alarm()
        

[ALERT] fall-down has just occurred!
fall detected between 1 and 2
self.buffer:  [3, 1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 1, 3, 2, 1, 1, 2, 3, 1]
--------
[ALERT] fall-down has just occurred!
fall detected between 0 and 1
self.buffer:  [1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 1, 1, 3, 2, 1, 1, 2, 3, 1, 2]
--------
[ALERT] fall-down has just occurred!
fall detected between 1 and 3
self.buffer:  [3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 1, 2, 2, 3, 2, 3]
--------
[ALERT] fall-down has just occurred!
fall detected between 0 and 2
self.buffer:  [1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 1, 2, 2, 3, 2, 3, 2]
--------
[ALERT] fall-down has just occurred!
fall detected between 0 and 1
self.buffer:  [1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 1, 2, 2, 3, 2, 3, 2, 1]
--------
[ALERT] fall-down has just occurred!
fall detected between 5 and 8
self.buffer:  [3, 2, 2, 1, 2, 1, 1, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1]
--------
[ALERT] fall-down has just occurred!
fall detected between 4 and 7
self.buffer:  [2, 2, 1, 2, 